# 유사도 높은 콘텐츠 추천 프로그램 
- munpia_novel_data 사용

# 데이터 형태 확인

In [6]:
import pandas as pd

df = pd.read_csv("munpia_novel_data.csv", encoding ="UTF-8-SIG")

df.head()

,id,title,author,intro,tags,genre,series_count,view_count,recommend_count,link
0,0,선독점 천조국은 이제 제겁니다,코알라,"캘리포니아, 천조국, 세계는 이제 제겁니다.","경영,농사,빙의,천재,환생,투자가,검은머리,먼치킨,나노머신,전설","현대판타지, 퓨전",95 회,"1,035,408","31,429",https://novel.munpia.com/409689
1,1,선독점 아포칼립스에 집을 숨김,로드워리어,"대충 세상은 망했고,\n나는 나대로 살아야지.\n\n물론 럭셔리하고 고져스하게.",NaN,"현대판타지, 퓨전",520 회,"7,033,600","392,241",https://novel.munpia.com/342817
2,2,선독점 회귀한 복학생이 건축 명문대 만듦,방구석리,과로사 후 지잡대 복학생 시절로 회귀한 건축 고인물. 편입을 위해 닥치는대로 작업하...,"현대판타지,드라마,사이다,이능력,회귀,건축가,디자이너,학생,능력자,먼치킨","현대판타지, 판타지",187 회,"1,038,449","28,943",https://novel.munpia.com/398144
3,3,선독점 역천무림,라바테라,제갈세가 시종의 몸에 빙의했다.\n적당히 꿀빨며 살려고 했는데... 아무래도 뭔가 ...,"무협,빙의,하렘,얀데레",무협,162 회,"1,163,881","27,819",https://novel.munpia.com/405796
4,4,선독점 고아출신 포수가 매의 눈을 가짐,키친캐비닛,"공이 너무 잘 보인다. 이번 공은 (4, 1) 몸쪽 낮은 코스.\n어떻게 지평좌표계...","현대판타지,스포츠,천재,회귀,운동선수,야구","스포츠, 현대판타지",112 회,"544,534","14,732",https://novel.munpia.com/412821


In [5]:
import chardet

with open('munpia_novel_data.csv', 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

df = pd.read_csv("munpia_novel_data.csv", encoding=encoding)
print(result)

{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}


In [7]:
df.columns

Index(['id', 'title', 'author', 'intro', 'tags', 'genre', 'series_count',
       'view_count', 'recommend_count', 'link'],
      dtype='object')

In [8]:
df.index

RangeIndex(start=0, stop=13945, step=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13945 entries, 0 to 13944
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               13945 non-null  int64 
 1   title            13945 non-null  object
 2   author           13945 non-null  object
 3   intro            13945 non-null  object
 4   tags             6078 non-null   object
 5   genre            13945 non-null  object
 6   series_count     13945 non-null  object
 7   view_count       13945 non-null  object
 8   recommend_count  13945 non-null  object
 9   link             13945 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.1+ MB


# 데이터 유사성 확인 중요도 결정

- id           = 작품 고유 번호 중요도 낮음

- title        = 유사도에 높은 관여를 하는 오브젝트로 결정

- author       = 유사도에 적은 관여를 하는  오브젝트로 결정

- intro        = 유사도에 높은 관여를 하는 오브젝트로 결정

- tags         = 유사도에 높은 관여를 하는 오브젝트로 결정

- genre        = 유사도에 높은 관여를 하는 오브젝트로 결정

- series_count = 유사도에 적은 관여를 함.

- view_count   = 유사도와 적은 관여를 함.

- recommend_count = 유사도와 관련이 높은 항목

- link         = 유사도와 관련 없음 

In [1]:
# 사용 모듈

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack



# 유사도 확인 프로그램 코드 작성
- TF-idf와 KNN 모델 사용    

In [4]:
# 데이터 불러오기
data = pd.read_csv('munpia_novel_data.csv')

# 필요 컬럼 선택
data = data[['title', 'author', 'intro', 'tags', 'genre']]
data.drop_duplicates(subset=['title'], inplace=True)  # 중복된 제목 제거

# 결측치 제거
data.dropna(subset=['intro'], inplace=True)

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = vectorizer.fit_transform(data['intro'])

# KNN 모델 학습
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(tfidf_matrix)

# 추천 함수 정의
def recommend_webtoon(title, num_recommendations=10):
    # 입력된 제목의 인덱스 찾기
    if title not in data['title'].values:
        print(f"'{title}' 제목의 웹툰을 찾을 수 없습니다.")
        return
    
    idx = data.index[data['title'] == title][0]
    tfidf_vector = tfidf_matrix[idx]
    
    # 유사한 웹툰 찾기
    distances, indices = knn_model.kneighbors(tfidf_vector, n_neighbors=num_recommendations + 1)
    
    # 결과 출력
    print(f"'{title}'와(과) 유사한 웹툰 추천 결과:")
    for i in range(1, len(indices.flatten())):
        similar_idx = indices.flatten()[i]
        similar_title = data.iloc[similar_idx]['title']
        similarity_score = 1 - distances.flatten()[i]
        print(f"{i}. {similar_title} (유사도: {similarity_score:.2f})")

# 테스트
title_to_recommend = input("추천받고 싶은 웹툰의 제목을 입력하세요: ")
recommend_webtoon(title_to_recommend)


'선독점 전지적 독자 시점'와(과) 유사한 웹툰 추천 결과:
1. 선독점 코딩의 신 IT재벌 되다 (유사도: 0.56)
2. 선독점 나 혼자만 강화술사 (유사도: 0.55)
3. 선독점 스킬 줍는 천재 기사 (유사도: 0.54)
4. 선독점 대마도사 서자는 전생 플레이어 (유사도: 0.44)
5. 선독점 실전된 무공으로 먼치킨 (유사도: 0.42)
6. 선독점 나 혼자 게이트 속 세상을 알고 있다. (유사도: 0.40)
7. 선독점 다차원의 강령술사 (유사도: 0.39)
8. 선독점 킬 더 히어로 (유사도: 0.37)
9. 선독점 미친 소드마스터들의 구원자 (유사도: 0.35)
10. 선독점 멸망한 세계의 커뮤니티 사용법 (유사도: 0.35)


# 목록별 가중치 설정    

In [ ]:
# 데이터 불러오기
data = pd.read_csv('munpia_novel_data.csv')

# 필요 컬럼 선택
data = data[['title', 'author', 'intro', 'tags', 'genre']]
data.drop_duplicates(subset=['title'], inplace=True)  # 중복된 제목 제거

# 결측치 제거
data.dropna(subset=['intro'], inplace=True)

# 가중치 설정 (예: title: 0.2, author: 0.1, intro: 0.5, tags: 0.1, genre: 0.1)
weight_title = 0.1
weight_author = 0.1
weight_intro = 0.3
weight_tags = 0.3
weight_genre = 0.2

# 각 컬럼에 대해 TF-IDF 벡터화
vectorizer_title = TfidfVectorizer(stop_words='english', max_features=500)
vectorizer_author = TfidfVectorizer(stop_words='english', max_features=500)
vectorizer_intro = TfidfVectorizer(stop_words='english', max_features=3000)
vectorizer_tags = TfidfVectorizer(stop_words='english', max_features=3000)
vectorizer_genre = TfidfVectorizer(stop_words='english', max_features=2000)

tfidf_title = vectorizer_title.fit_transform(data['title']) * weight_title
tfidf_author = vectorizer_author.fit_transform(data['author'].fillna('')) * weight_author
tfidf_intro = vectorizer_intro.fit_transform(data['intro']) * weight_intro
tfidf_tags = vectorizer_tags.fit_transform(data['tags'].fillna('')) * weight_tags
tfidf_genre = vectorizer_genre.fit_transform(data['genre'].fillna('')) * weight_genre

# 각 TF-IDF 벡터 결합
tfidf_matrix = hstack([tfidf_title, tfidf_author, tfidf_intro, tfidf_tags, tfidf_genre])

# KNN 모델 학습
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(tfidf_matrix)

# 추천 함수 정의
def recommend_webtoon(title, num_recommendations=10):
    # 입력된 제목의 인덱스 찾기
    if title not in data['title'].values:
        print(f"'{title}' 제목의 웹툰을 찾을 수 없습니다.")
        return
    
    idx = data.index[data['title'] == title][0]
    tfidf_vector = tfidf_matrix[idx]
    
    # 유사한 웹툰 찾기
    distances, indices = knn_model.kneighbors(tfidf_vector, n_neighbors=num_recommendations + 1)
    
    # 결과 출력
    print(f"'{title}'와(과) 유사한 웹툰 추천 결과:")
    for i in range(1, len(indices.flatten())):
        similar_idx = indices.flatten()[i]
        similar_title = data.iloc[similar_idx]['title']
        similarity_score = 1 - distances.flatten()[i]
        print(f"{i}. {similar_title} (유사도: {similarity_score:.2f})")

# 테스트
title_to_recommend = input("추천받고 싶은 웹툰의 제목을 입력하세요: ")
recommend_webtoon(title_to_recommend)

# 설명: 제목(title), 작가(author), 소개(intro), 태그(tags), 장르(genre)별로 가중치를 적용하여 유사도를 계산합니다.
# 이를 통해 중요한 요소에 가중치를 반영해 더욱 정교한 추천 결과를 제공합니다.


'선독점 전지적 독자 시점'와(과) 유사한 웹툰 추천 결과:
1. 선독점 코딩의 신 IT재벌 되다 (유사도: 0.73)
2. 선독점 마법천재 마술방송 (유사도: 0.60)
3. 선독점 감독보다 영화사 대표님 (유사도: 0.60)
4. 선독점 내 아공간이 지구와 연결되어있다 (유사도: 0.55)
5. 선독점 먼치킨의 정석 (유사도: 0.55)
6. 선독점 천상계 지갑전사 (유사도: 0.55)
7. 선독점 책보고 가라 (유사도: 0.55)
8. 선독점 정자부터 잘생김 (유사도: 0.55)
9. 데우스 큐브(DEUS CUBE) (유사도: 0.53)
10. 선독점 아포칼립스로 억만장자 (유사도: 0.53)


- 가중치 변경시 유사도가 올라감. 현재 최적 가중치 
- weight_title = 0.1
- weight_author = 0.1
- weight_intro = 0.3
- weight_tags = 0.3
- weight_genre = 0.2

# TF-idf에 한국어 불용어 적용 안된 것을 확인 불용어 적용    

In [ ]:
# 데이터 불러오기
data = pd.read_csv('munpia_novel_data.csv')

# 필요 컬럼 선택
data = data[['title', 'author', 'intro', 'tags', 'genre']]
data.drop_duplicates(subset=['title'], inplace=True)  # 중복된 제목 제거

# 결측치 제거
data.dropna(subset=['intro'], inplace=True)

# 한국어 불용어 리스트를 리스트로 설정
stop_words = ["를", "의", "선", "독점", "선독점", "독점작", "이번", "들", "등", "수", "이", "부", "판", "뿐", "여자", "남자", "그", "것", "나", "그녀", "속", "시작", "내", "거", "중"]

# 가중치 설정 (예: title: 0.1, author: 0.1, intro: 0.3, tags: 0.3, genre: 0.2)
weight_title = 0.1
weight_author = 0.1
weight_intro = 0.3
weight_tags = 0.3
weight_genre = 0.2

# 각 컬럼에 대해 TF-IDF 벡터화
vectorizer_title = TfidfVectorizer(stop_words=stop_words, max_features=500)
vectorizer_author = TfidfVectorizer(stop_words=stop_words, max_features=500)
vectorizer_intro = TfidfVectorizer(stop_words=stop_words, max_features=3000)
vectorizer_tags = TfidfVectorizer(stop_words=stop_words, max_features=3000)
vectorizer_genre = TfidfVectorizer(stop_words=stop_words, max_features=2000)

tfidf_title = vectorizer_title.fit_transform(data['title']) * weight_title
tfidf_author = vectorizer_author.fit_transform(data['author'].fillna('')) * weight_author
tfidf_intro = vectorizer_intro.fit_transform(data['intro']) * weight_intro
tfidf_tags = vectorizer_tags.fit_transform(data['tags'].fillna('')) * weight_tags
tfidf_genre = vectorizer_genre.fit_transform(data['genre'].fillna('')) * weight_genre

# 각 TF-IDF 벡터 결합
tfidf_matrix = hstack([tfidf_title, tfidf_author, tfidf_intro, tfidf_tags, tfidf_genre])

# KNN 모델 학습
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(tfidf_matrix)

# 추천 함수 정의
def recommend_webtoon(title, num_recommendations=10):
    # 입력된 제목의 인덱스 찾기
    if title not in data['title'].values:
        print(f"'{title}' 제목의 웹툰을 찾을 수 없습니다.")
        return
    
    idx = data.index[data['title'] == title][0]
    tfidf_vector = tfidf_matrix[idx]
    
    # 유사한 웹툰 찾기
    distances, indices = knn_model.kneighbors(tfidf_vector, n_neighbors=num_recommendations + 1)
    
    # 결과 출력
    print(f"'{title}'와(과) 유사한 웹툰 추천 결과:")
    for i in range(1, len(indices.flatten())):
        similar_idx = indices.flatten()[i]
        similar_title = data.iloc[similar_idx]['title']
        similarity_score = 1 - distances.flatten()[i]
        print(f"{i}. {similar_title} (유사도: {similarity_score:.2f})")

# 테스트
title_to_recommend = input("추천받고 싶은 웹툰의 제목을 입력하세요: ")
recommend_webtoon(title_to_recommend)

# 설명: 각 컬럼에 대해 불용어 리스트를 적용하여 TF-IDF 벡터화를 수행합니다.
# 제목(title), 작가(author), 소개(intro), 태그(tags), 장르(genre)별로 가중치를 적용하여 유사도를 계산합니다.


'선독점 전지적 독자 시점'와(과) 유사한 웹툰 추천 결과:
1. 선독점 코딩의 신 IT재벌 되다 (유사도: 0.74)
2. 선독점 감독보다 영화사 대표님 (유사도: 0.55)
3. 선독점 마법천재 마술방송 (유사도: 0.55)
4. 선독점 내 아공간이 지구와 연결되어있다 (유사도: 0.52)
5. 선독점 미친 소드마스터들의 구원자 (유사도: 0.51)
6. 선독점 대마도사 서자는 전생 플레이어 (유사도: 0.51)
7. 선독점 실전된 무공으로 먼치킨 (유사도: 0.50)
8. 선독점 신이 내린 방송천재 (유사도: 0.50)
9. 3차대전 (유사도: 0.50)
10. 선독점 천상계 지갑전사 (유사도: 0.50)
